In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/processed.csv')

In [3]:
### Season to test results

N = 2021

In [4]:
params_to_drop = ['season', 'round', 'podium', 'driver_points_from']

In [5]:
df = data.copy()

train = df[df.season < N]

scaler = StandardScaler()
X_train = train.drop(params_to_drop, axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

y_train = np.asarray(train.driver_points_from.values)

In [6]:
X_train.columns

Index(['q_delta', 'starting_grid', 'driver_points_before',
       'constructor_points_before', 'driver_points_per', 'points_percentage',
       'driver_last_3', 'constructor_last_3', 'driver_adrian_sutil',
       'driver_alexander_albon',
       ...
       'circuit_id_rodriguez', 'circuit_id_sepang', 'circuit_id_shanghai',
       'circuit_id_silverstone', 'circuit_id_sochi', 'circuit_id_spa',
       'circuit_id_suzuka', 'circuit_id_villeneuve', 'circuit_id_yas_marina',
       'circuit_id_zandvoort'],
      dtype='object', length=112)

In [7]:
comparison_dict ={'model':[],
                  'criterion': [],
                  'max_features': [],
                  'max_depth': [],
                  'score': []
                  }

In [8]:
def score_regression(X_train, y_train, model):
    correct_predictions = 0
    
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(params_to_drop, axis=1)
        y_test = test.driver_points_from

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns=['predicted_points'])
        prediction_df['actual_points'] = y_test.reset_index(drop=True)
        prediction_df.sort_values('predicted_points', ascending=False, inplace=True)

        prediction_df['predicted_winner'] = prediction_df.predicted_points.map(lambda x: 1 if x == prediction_df.predicted_points.max() else 0)
        prediction_df['actual_winner'] = prediction_df.actual_points.map(lambda x: 1 if x == prediction_df.actual_points.max() else 0)

        correct_predictions += precision_score(prediction_df.actual_winner, prediction_df.predicted_winner)
        
    return correct_predictions / df[df.season == N]['round'].nunique()

In [9]:
# Random Forest Regressor

params={'criterion': ['squared_error'],
        'max_features': [0.8, 1.0],
        'max_depth': list(map(int, (np.linspace(5, 55, 26))))
        }

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:

            model_params = (criterion, max_features, max_depth)
            model = RandomForestRegressor(criterion = criterion, max_features = max_features, max_depth = max_depth)
            model.fit(X_train, y_train)
            
            score = score_regression(X_train, y_train, model)

            comparison_dict['model'].append('random_forest_regressor')
            comparison_dict['criterion'].append(criterion)
            comparison_dict['max_features'].append(max_features)
            comparison_dict['max_depth'].append(max_depth)
            comparison_dict['score'].append(score)

In [10]:
comparison_df = pd.DataFrame(comparison_dict)

chosen_inputs = comparison_df.query('score > 0.5').sort_values('score', ascending=False)


chosen_inputs.to_csv(path+'parameters/rf_regressor.csv', index=False)

chosen_inputs.head()

,model,criterion,max_features,max_depth,score
33,random_forest_regressor,squared_error,1.0,19,0.631579
17,random_forest_regressor,squared_error,0.8,39,0.578947
32,random_forest_regressor,squared_error,1.0,17,0.578947
30,random_forest_regressor,squared_error,1.0,13,0.578947
28,random_forest_regressor,squared_error,1.0,9,0.578947
